# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 1 2022 8:30AM,240904,10,0085817485,ISDIN Corporation,Released
1,Jul 1 2022 8:30AM,240904,10,0085817488,ISDIN Corporation,Released
2,Jul 1 2022 8:30AM,240904,10,0085817493,ISDIN Corporation,Released
3,Jul 1 2022 8:30AM,240904,10,0085817491,ISDIN Corporation,Released
4,Jul 1 2022 8:30AM,240904,10,0085817495,ISDIN Corporation,Released
5,Jul 1 2022 8:30AM,240904,10,0085817496,ISDIN Corporation,Released
6,Jul 1 2022 8:30AM,240904,10,0085817497,ISDIN Corporation,Released
7,Jul 1 2022 8:30AM,240904,10,0085817501,ISDIN Corporation,Released
8,Jul 1 2022 8:30AM,240904,10,0085817502,ISDIN Corporation,Released
9,Jul 1 2022 8:30AM,240900,10,8214321,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,240913,Released,1
28,240914,Released,2
29,240915,Released,1
30,240916,Released,1
31,240918,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240913,NaN,NaN,1.0
240914,NaN,NaN,2.0
240915,NaN,NaN,1.0
240916,NaN,NaN,1.0
240918,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240804,0.0,2.0,1.0
240818,0.0,1.0,0.0
240826,3.0,8.0,7.0
240831,0.0,0.0,1.0
240847,28.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240804,0,2,1
240818,0,1,0
240826,3,8,7
240831,0,0,1
240847,28,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240804,0,2,1
1,240818,0,1,0
2,240826,3,8,7
3,240831,0,0,1
4,240847,28,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240804,,2,1
1,240818,,1,
2,240826,3,8,7
3,240831,,,1
4,240847,28,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 1 2022 8:30AM,240904,10,ISDIN Corporation
9,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.
10,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.
11,Jul 1 2022 8:30AM,240910,10,"Citieffe, Inc."
12,Jul 1 2022 8:30AM,240898,10,Emerginnova
13,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC"
18,Jul 1 2022 8:14AM,240918,16,Sartorius Bioprocess Solutions
21,Jul 1 2022 8:14AM,240918,16,Sartorius Lab Products and Service
24,Jul 1 2022 7:53AM,240916,19,ACG North America LLC
25,Jul 1 2022 7:45AM,240915,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 1 2022 8:30AM,240904,10,ISDIN Corporation,,,9
1,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.,,,2
2,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.,,,1
3,Jul 1 2022 8:30AM,240910,10,"Citieffe, Inc.",,,1
4,Jul 1 2022 8:30AM,240898,10,Emerginnova,,,1
5,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC",,,4
6,Jul 1 2022 8:14AM,240918,16,Sartorius Bioprocess Solutions,,,6
7,Jul 1 2022 8:14AM,240918,16,Sartorius Lab Products and Service,,,6
8,Jul 1 2022 7:53AM,240916,19,ACG North America LLC,,,1
9,Jul 1 2022 7:45AM,240915,19,"Emersa Waterbox, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 8:30AM,240904,10,ISDIN Corporation,9,,
1,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.,2,,
2,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.,1,,
3,Jul 1 2022 8:30AM,240910,10,"Citieffe, Inc.",1,,
4,Jul 1 2022 8:30AM,240898,10,Emerginnova,1,,
5,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC",4,,
6,Jul 1 2022 8:14AM,240918,16,Sartorius Bioprocess Solutions,6,,
7,Jul 1 2022 8:14AM,240918,16,Sartorius Lab Products and Service,6,,
8,Jul 1 2022 7:53AM,240916,19,ACG North America LLC,1,,
9,Jul 1 2022 7:45AM,240915,19,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 8:30AM,240904,10,ISDIN Corporation,9,,
1,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.,2,,
2,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.,1,,
3,Jul 1 2022 8:30AM,240910,10,"Citieffe, Inc.",1,,
4,Jul 1 2022 8:30AM,240898,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 8:30AM,240904,10,ISDIN Corporation,9.0,NaN,NaN
1,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.,2.0,NaN,NaN
2,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.,1.0,NaN,NaN
3,Jul 1 2022 8:30AM,240910,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Jul 1 2022 8:30AM,240898,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 8:30AM,240904,10,ISDIN Corporation,9.0,0.0,0.0
1,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.,2.0,0.0,0.0
2,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.,1.0,0.0,0.0
3,Jul 1 2022 8:30AM,240910,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Jul 1 2022 8:30AM,240898,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1686295,19.0,0.0,0.0
15,240847,0.0,1.0,28.0
16,1445330,14.0,2.0,0.0
19,2167952,32.0,2.0,0.0
20,722590,16.0,8.0,3.0
22,722734,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1686295,19.0,0.0,0.0
1,15,240847,0.0,1.0,28.0
2,16,1445330,14.0,2.0,0.0
3,19,2167952,32.0,2.0,0.0
4,20,722590,16.0,8.0,3.0
5,22,722734,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,19.0,0.0,0.0
1,15,0.0,1.0,28.0
2,16,14.0,2.0,0.0
3,19,32.0,2.0,0.0
4,20,16.0,8.0,3.0
5,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,15,Released,0.0
2,16,Released,14.0
3,19,Released,32.0
4,20,Released,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,22
Status,,,,,,
Completed,0.0,28.0,0.0,0.0,3.0,0.0
Executing,0.0,1.0,2.0,2.0,8.0,0.0
Released,19.0,0.0,14.0,32.0,16.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,22
0,Completed,0.0,28.0,0.0,0.0,3.0,0.0
1,Executing,0.0,1.0,2.0,2.0,8.0,0.0
2,Released,19.0,0.0,14.0,32.0,16.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,22
0,Completed,0.0,28.0,0.0,0.0,3.0,0.0
1,Executing,0.0,1.0,2.0,2.0,8.0,0.0
2,Released,19.0,0.0,14.0,32.0,16.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()